In [0]:
df_oct = spark.table("workspace.ecommerce.events_oct")
display(df_oct)

# Top 5 Products By Revenue

In [0]:
top5_products = (
    df_oct.groupBy("product_id")
    .agg({"price": "sum"})
    .withColumnRenamed("sum(price)", "total_revenue")
    .orderBy("total_revenue", ascending=False)
    .limit(5)
)
display(top5_products)

# Total_Per_User

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window = Window.partitionBy("user_id").orderBy("event_time")
total_per_user = df_oct.withColumn("cumulative_events", F.count("*").over(window))
display(total_per_user)

# Conversion_Rate_By_Category

In [0]:
conversion_rate_by_category = (
    df_oct.groupBy("category_code")
    .agg(
        (F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)) / F.count("*"))
        .alias("conversion_rate")
    )
)
display(conversion_rate_by_category)